In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np, pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import brier_score_loss
from sklearn.calibration import CalibratedClassifierCV
from sklearn.pipeline import Pipeline

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture as GMM
from sklearn.mixture import BayesianGaussianMixture as BGMM

from sklearn.decomposition import PCA, KernelPCA

from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

cols = [c for c in train.columns if c not in ['id', 'target', 'wheezy-copper-turtle-magic']]

In [ ]:

oof_svc = np.zeros(len(train))


preds = np.zeros(len(test))

for i in tqdm_notebook(range(512)):

    train2 = train[train['wheezy-copper-turtle-magic']==i]
    test2 = test[test['wheezy-copper-turtle-magic']==i]
    idx1 = train2.index; idx2 = test2.index
    train2.reset_index(drop=True,inplace=True)

    data = pd.concat([pd.DataFrame(train2[cols]), pd.DataFrame(test2[cols])])
    #data2 = VarianceThreshold(threshold=2).fit_transform(data[cols])
    
    pipe = Pipeline([('vt', VarianceThreshold(threshold=2)), ('scaler', StandardScaler())])
    data2 = pipe.fit_transform(data[cols])
    
    
    #gmm = GMM(n_components=4,covariance_type='full',random_state=42).fit(data2)
    gmm = BGMM(n_components=4,covariance_type='full',random_state=42,n_init=10).fit(data2)
    #gmm = GMM(n_components=4,covariance_type='full',random_state=42,init_params='random').fit(data2)
    yclust = gmm.predict(data2)
    
    #plt.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis');

    data2=np.hstack((data2,yclust.reshape(-1,1)))

    train3 = data2[:train2.shape[0]]; test3 = data2[train2.shape[0]:]

    skf = StratifiedKFold(n_splits=11, random_state=42)
    for train_index, test_index in skf.split(train2, train2['target']):

        

        clf_svc = QuadraticDiscriminantAnalysis(0.5)
        #clf_svc = SVC(probability=True,kernel='poly',degree=4,gamma='auto')
        clf_svc.fit(train3[train_index,:],train2.loc[train_index]['target'])
        oof_svc[idx1[test_index]] = clf_svc.predict_proba(train3[test_index,:])[:,1]
        preds[idx2] += clf_svc.predict_proba(test3)[:,1] / skf.n_splits
        
        

auc_svc = roc_auc_score(train['target'], oof_svc)

print(f'AUC: {auc_svc:.5}')


In [ ]:
preds_svc = preds.copy()

In [ ]:
#preds = preds_svc.copy()

In [ ]:
pd.DataFrame(preds).hist(bins=100)

In [ ]:
for itr in range(1):
    test['target'] = preds
    test.loc[test['target'] > 0.94, 'target'] = 1 # initial 94
    test.loc[test['target'] < 0.06, 'target'] = 0 # initial 06
    usefull_test = test[(test['target'] == 1) | (test['target'] == 0)]
    new_train = pd.concat([train, usefull_test]).reset_index(drop=True)
    print(usefull_test.shape[0], "Test Records added for iteration : ", itr)
    new_train.loc[oof_svc > 0.98, 'target'] = 1 # initial 98
    new_train.loc[oof_svc < 0.02, 'target'] = 0 # initial 02
    oof_final = np.zeros(len(train))
    preds = np.zeros(len(test))
    for i in tqdm_notebook(range(512)):

        train2 = new_train[new_train['wheezy-copper-turtle-magic']==i]
        test2 = test[test['wheezy-copper-turtle-magic']==i]
        idx1 = train[train['wheezy-copper-turtle-magic']==i].index
        idx2 = test2.index
        train2.reset_index(drop=True,inplace=True)

        data = pd.concat([pd.DataFrame(train2[cols]), pd.DataFrame(test2[cols])])
        pipe = Pipeline([('vt', VarianceThreshold(threshold=2)), ('scaler', StandardScaler())])
        data2 = pipe.fit_transform(data[cols])
        
        
        gmm = BGMM(n_components=4,covariance_type='full',random_state=42,n_init=10).fit(data2)
        yclust = gmm.predict(data2)

        #plt.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis');

        data2=np.hstack((data2,yclust.reshape(-1,1)))
        
        
        train3 = data2[:train2.shape[0]]
        test3 = data2[train2.shape[0]:]
    
        skf = StratifiedKFold(n_splits=11,shuffle=True)
        for train_index, test_index in skf.split(train2, train2['target']):
            oof_test_index = [t for t in test_index if t < len(idx1)]
            
            clf = QuadraticDiscriminantAnalysis(0.5)
            clf.fit(train3[train_index,:],train2.loc[train_index]['target'])
            if len(oof_test_index) > 0:
                oof_final[idx1[oof_test_index]] = clf.predict_proba(train3[oof_test_index,:])[:,1]
            preds[idx2] += clf.predict_proba(test3)[:,1] / skf.n_splits
        
    auc = roc_auc_score(train['target'], oof_final)
    
    print(f'AUC: {auc:.5}')

In [ ]:
sub = pd.read_csv('../input/sample_submission.csv')
sub['target'] = preds
sub.to_csv('samplesubmission.csv', index=False)